In [1]:
from dotenv import load_dotenv
load_dotenv(override=True)

True

### [실습] 
- 1. Google Gemini 모델을 사용하여 텍스트 생성하고 응답 객체를 확인해보세요.
- 2. Google Gemini 모델을 사용하여, 챗 메시지 목록을 기반으로 텍스트 생성하고 응답 객체를 확인해보세요.
- 참고: https://python.langchain.com/docs/integrations/chat/google_generative_ai/

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import SystemMessage, HumanMessage

# OpenAI 모델을 사용하여 대화 생성
model = ChatGoogleGenerativeAI(
	model="gemini-2.5-flash-lite", 
	temperature=0.3
)

response = model.invoke("Why do parrots have colorful feathers?")
print('1. response : ', response)

system_msg = SystemMessage(content="당신은 영어를 한국어로 번역하는 AI 어시스턴스 입니다. 사용자가 요청하는 영어문자를 원문 그대로 한국어로 번역해 주세요.")
human_msg = HumanMessage(content="Translate the word 'I Love Programming' to Korean")

# messages = [
# 	(
# 		"system",
# 		"You are a helpful assistant that translates English to French. Translate the user sentence.",
# 	),
# 	(
# 		"human", 
# 		"I love programming."
# 	),
# ]

messages = [system_msg, human_msg]

ai_msg = model.invoke(messages)
print('2. ai_msg : ', ai_msg)
print('content : ', ai_msg.content)
print('text : ', ai_msg.text)


response :  content="Parrots have colorful feathers for a variety of fascinating reasons, primarily related to **survival and reproduction**. Here's a breakdown of the key factors:\n\n**1. Camouflage:**\n\n*   **Blending in with their environment:** Many parrot species live in lush, tropical rainforests with a vibrant array of green foliage, flowers, and fruits. Their bright colors, especially greens, can help them blend seamlessly into this environment, making them harder for predators to spot. This is a crucial survival mechanism.\n*   **Species-specific camouflage:** While it might seem counterintuitive for bright colors to aid camouflage, the specific patterns and hues can be very effective in breaking up the parrot's outline against the complex backdrop of leaves and branches.\n\n**2. Communication and Social Signaling:**\n\n*   **Mate attraction:** This is arguably one of the most significant reasons for vibrant coloration. In many species, brighter and more elaborate plumage ind

### 프롬프트 템플릿
- 역할
	- 프롬프트 템플릿은 일관된 입력 형식을 제공하기 위해 사용
	1. 사용자의 입력과 파라미터를 언어 모델이 이해할 수 있는 형태로 변환하는 도구
	2. 언어 모델에게 전달할 지시문을 만드는 틀
- 변수를 포함하여 동적 프롬프트 생성이 가능
	1. 모든 템플릿은 dictionary(사전)형태의 입력을 받아서 처리
	2. 출력은 promptValue형태로 변환되며, 이는 문자열이나 메시지 리스트로 변환 가능하다.


1. 문자열 프롬프트 템플릿 String PromptTemplate
	- 가장 기본적인 형태
	- 단일 문자 열을 형식화하는데 사용
	- 메서드
		- invoke() : 템플릿에 값을 전달

In [13]:
from langchain_core.prompts import PromptTemplate

# 템플릿 생성
# "{topic}에 대한 이야기를 해줘"라는 템플릿을 사용하여 topic이라는 변수를 포함하는 프롬프트 생성
template = PromptTemplate.from_template("{topic}에 대한 이야기를 해줘")

# 템플릿 사용
# "고양이"라는 주제를 사용하여 프롬프트 생성
# invoke 메서드를 통해 템플릿에 값을 전달
prompt = template.invoke({'topic': '고양이'})

# 템플릿 출력
prompt

StringPromptValue(text='고양이에 대한 이야기를 해줘')

2. 채팅 프롬프트 템플릿
- 여러 메시지를 포함하는 대화형 탬플릿을 만들 때 사용

In [16]:
from langchain_core.prompts import ChatPromptTemplate

template = ChatPromptTemplate.from_messages([
	("system", "당신은 도움이 되는 비서입니다."),
	("user", "{subject}에 대해 설명해주세요.")
])

prompt = template.invoke({'subject': '인공지능'})

print(prompt)

prompt.messages

messages=[SystemMessage(content='당신은 도움이 되는 비서입니다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='인공지능에 대해 설명해주세요.', additional_kwargs={}, response_metadata={})]


[SystemMessage(content='당신은 도움이 되는 비서입니다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='인공지능에 대해 설명해주세요.', additional_kwargs={}, response_metadata={})]

## 출력 파서 (Output Parser)
- 역할과 기능
	- 모델의 텍스트 출력을 구조화 된 데이터로 변환
	- 채팅 모델과 LLM의 출력을 정규화
	- 다운스트림 작업을 위한 데이터 형식 변환
- 사용 시 고려사항
	- openAI function calling과 같은 기능이 있는 경우, 해당 기능을 우선 사용
- 종류
	- StrOutParser
	- 구조화 된 출력 : with_structured_output 메소드

In [ ]:
### StrOutputParser

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 프롬프트 템플릿 설정
prompt = PromptTemplate.from_template("도시 {city}의 특징을 알려주세요")

# 모델 정의
model = ChatOpenAI(model='gpt-4.1-mini')

# 기본 문자열 파서 사용
parser = StrOutputParser()

# LCEL체인 구성
chain = prompt | model | parser

result = chain.invoke({"city": "서울"})



In [17]:
### 구조화된 출력 - with_structured_output()

from pydantic import BaseModel, Field
from typing import List

# 출력 스키마
class NewsInfo(BaseModel):
    """뉴스 기사에 대한 정보"""
    title: str = Field(description="기사 제목")
    content: str = Field(description="기사 내용 (원문 그대로 추출)")
    press: str = Field(description="언론사")
    author: str = Field(description="작성자 이름")
    published_date: List[str] = Field(description="작성일(예: 2023년 1월 1일)")

# 뉴스 기사 예제 데이터
news_article = """
"복잡한 납세 등 AI가 돕는다"…정부, 초거대 AI 공공서비스 개발
입력2025.06.20. 오후 12:00  수정2025.06.20. 오후 1:39 

[서울=뉴시스]윤현성 기자 = 정부가 공공분야에 도입할 초거대 인공지능(AI) 기반 국민 편의 서비스 본격 개발에 나선다. 세금 납부 시 생성형 AI 챗봇을 통해 어려운 세무 용어 등에 대한 설명·상담을 듣거나, 대량의 민원업무도 생성형 AI 기반 분석으로 빠르게 답변·응대하는 등의 형태다.

과학기술정보통신부와 디지털플랫폼정부위원회는 2025년도 '초거대 AI 서비스 개발 지원' 사업을 본격 추진하기 위해 수행기업 공모를 실시한다고 20일 밝혔다. 이 사업은 공공분야에 초거대 AI를 도입·확산하고 이를 통해 행정 효율화, 대국민 서비스 혁신, 사회현안 해결이 가능한 서비스 개발을 목표로 추진된다.

올해는 다양한 공공분야에서 초거대 AI 기술을 통해 국민이 체감할 수 있고 실질적인 변화를 가져올 수 있는 과제를 중점적으로 발굴했다. 중앙부처·지자체·공공기관을 대상으로 1~2월에 과제 공모를 추진했으며 총 5개 과제가 선정됐다.

국민권익위원회의 '생성형 AI 기반 국민소통·민원분석 체계 구축'은 국민소통시스템에 생성형 AI 기반 민원분석 체계를 도입해 민원처리 행정 효율화와 답변품질을 향상시킨다. 이를 통해 대량의 민원업무를 신속·효율적으로 대응해 민원업무의 효율성을 증대하고 국민의 신뢰도 향상에도 기여할 것으로 기대된다.

국세청의 '생성형 AI 기반 국세 상담 지원 서비스'는 납세자가 홈택스 이용 시 전자신고 관련한 문의사항을 즉시 해소할 수 있는 실시간 상담 서비스를 제공한다. 홈택스에 상담전용 AI챗봇을 도입해 전화 상담 시 발생하는 장시간 대기 문제를 해결하고, 어려운 세무 용어 등으로 인한 불편사항을 개선할 예정이다.

산업통상자원부의 '해외인증 공공특화 AI 에이전트 서비스'는 모바일 플랫폼, 소셜네트워크서비스 등 사용자 친화적인 모바일 기반 해외인증 특화 AI 에이전트 서비스를 제공한다. 중소기업이 겪는 수출 관련 애로사항인 해외 인증과 관련된 정보와 질의 응답을 AI기반으로 제공하여 손쉽게 확인할 수 있을 것으로 기대된다.

국민건강보험공단의 '에이전틱 AI기반 전국민 맞춤형 민원 상담 서비스'는 국민 생활과 편익에 직결되는 건강보험 민원 상담업무에 AI를 도입해 24시간 개인 맞춤형 민원 상담 서비스를 구현한다. 기존의 전화 상담 방식의 대기 시간 문제 등을 해소하고, 고객센터 집중 상담을 분산시켜 업무의 효율성을 향상시키는데 기여할 것으로 전망이다.

한국지역정보개발원의 '지방재정 지능화 서비스'는 e호조+, 지방재정365 등 지방재정서비스에 생성형 AI를 접목시켜 대국민, 공무원 등 각자의 요구에 부합하는 융복합 재정정보서비스 환경을 제공하고자 한다. 이 서비스가 도입되면 지방정부의 사회현안 해결을 위한 정책 수립의 적시성 향상 및 전문성 확보, 지자체 정보 접근성 강화로 대국민의 사회 참여 기회를 확대할 것으로 예상된다.

이번 사업은 19일 국민권익위원회와 국세청 과제의 민간 전문기업 조달 공고를 시작으로, 5개 과제별 서비스 개발지원 사업이 순차적으로 입찰공고될 예정이다.

김경만 과기정통부 인공지능기반정책관은 "선정된 과제에 대해 민·관 협력을 기반으로 행정 현장의 변화와 국민이 체감할 수 있는 성과가 창출·확산될 수 있도록 적극 지원할 계획"이라며 "개발된 서비스는 공공분야에서 행정업무의 효율성을 높이고 대국민 서비스 품질을 높이는 데 실질적으로 기여할 수 있도록 민간에서도 많은 관심을 가져주실 것을 부탁드린다"고 말했다.

이승현 디플정위 인공지능·플랫폼혁신국장은 "이번 사업은 노동, 복지, 민원 등 다양한 공공 분야에 AI를 도입·활용하는데 중요한 역할을 하고 있다"며 "올해도 AI를 활용해 사회문제를 해결하고, 대국민 서비스를 혁신적으로 개선할 수 있는 서비스가 개발되길 기대한다"고 전했다．

윤현성 기자(hsyhs@newsis.com)
"""

# 프롬프트 템플릿 생성 - 일관된 입력 형식 제공
prompt = PromptTemplate.from_template(
    """다음 뉴스 기사에서 관련 정보를 출력 스키마에 맞춰서 추출해주세요. 예시를 참고하세요.

    ## 뉴스 기사
    {news_content}

    ## 출력 스키마
    - title: 기사 제목
    - content: 기사 내용 (원문 그대로 추출)
    - press: 언론사
    - author: 작성자 이름. 반드시 '이름, 성' 순서로 출력!!!! (예: 길동,홍)
    - published_date: 작성일(예: 2023년 1월 1일)
    """
)

# 모델 생성 및 구조화된 출력 바인딩
model = ChatOpenAI(
    model='gpt-4.1-mini',
    temperature=0.3
)

# with_structured_output 메서드로 출력 스키마대로 출력이 된다.
structured_model = model.with_structured_output(NewsInfo)

# 프롬프트와 모델 체인 연결
chain = prompt | structured_model

# invoke()로 prompt 템플릿에 값 전달
result = chain.invoke({"news_content": news_article})

# 결과 출력
print(f"기사 제목: {result.title}")
print(f"언론사: {result.press}")
print(f"작성자: {result.author}")
print(f"작성일: {result.published_date}")

기사 제목: "복잡한 납세 등 AI가 돕는다"…정부, 초거대 AI 공공서비스 개발
언론사: 뉴시스
작성자: 현성,윤
작성일: ['2025년 6월 20일']


In [18]:
# 결과 JSON으로 출력
result.model_dump()

{'title': '"복잡한 납세 등 AI가 돕는다"…정부, 초거대 AI 공공서비스 개발',
 'content': '입력2025.06.20. 오후 12:00  수정2025.06.20. 오후 1:39 \n\n[서울=뉴시스]윤현성 기자 = 정부가 공공분야에 도입할 초거대 인공지능(AI) 기반 국민 편의 서비스 본격 개발에 나선다. 세금 납부 시 생성형 AI 챗봇을 통해 어려운 세무 용어 등에 대한 설명·상담을 듣거나, 대량의 민원업무도 생성형 AI 기반 분석으로 빠르게 답변·응대하는 등의 형태다.\n\n과학기술정보통신부와 디지털플랫폼정부위원회는 2025년도 \'초거대 AI 서비스 개발 지원\' 사업을 본격 추진하기 위해 수행기업 공모를 실시한다고 20일 밝혔다. 이 사업은 공공분야에 초거대 AI를 도입·확산하고 이를 통해 행정 효율화, 대국민 서비스 혁신, 사회현안 해결이 가능한 서비스 개발을 목표로 추진된다.\n\n올해는 다양한 공공분야에서 초거대 AI 기술을 통해 국민이 체감할 수 있고 실질적인 변화를 가져올 수 있는 과제를 중점적으로 발굴했다. 중앙부처·지자체·공공기관을 대상으로 1~2월에 과제 공모를 추진했으며 총 5개 과제가 선정됐다.\n\n국민권익위원회의 \'생성형 AI 기반 국민소통·민원분석 체계 구축\'은 국민소통시스템에 생성형 AI 기반 민원분석 체계를 도입해 민원처리 행정 효율화와 답변품질을 향상시킨다. 이를 통해 대량의 민원업무를 신속·효율적으로 대응해 민원업무의 효율성을 증대하고 국민의 신뢰도 향상에도 기여할 것으로 기대된다.\n\n국세청의 \'생성형 AI 기반 국세 상담 지원 서비스\'는 납세자가 홈택스 이용 시 전자신고 관련한 문의사항을 즉시 해소할 수 있는 실시간 상담 서비스를 제공한다. 홈택스에 상담전용 AI챗봇을 도입해 전화 상담 시 발생하는 장시간 대기 문제를 해결하고, 어려운 세무 용어 등으로 인한 불편사항을 개선할 예정이다.\n\n산업통상자원부의 \'해외인증 공공특화 AI 에이전트 서비스\'는 모바일 플랫폼, 소셜네트워크서비스 